**ЛАБОРАТОРНАЯ РАБОТА 9. ИСПОЛЬЗОВАНИЕ 
РАЗРАБОТАННОГО ПАЙПЛАЙНА ДЛЯ МНОГОМЕРНОЙ 
РЕГРЕССИИ**

**В качестве наборов ресурсов, я использую Real estate valuation data set.xlsx**

**Описание набора**

**Оценка недвижимости - это регрессионная задача. Набор рыночных исторических данных для оценки недвижимости собран в районе Синдиан, Нью-Тайбэй, Тайвань. Набор данных был случайным образом разделён на обучающую выборку (2/3 образцов) и тестовую выборку (1/3 образцов).**

**Переменные таблицы:**

**Ввод данных осуществляется следующим образом**

 **No - ID**
 
 **X1 transaction date - Дата транзакции X1 = дата сделки (например, 2013.250=март 2013, 2013.500=июнь 2013 и т. д.)**
 
 **X2 house age - Возраст дома X2 = возраст дома (единица измерения: год)**
 
 **X3 distance to the nearest MRT station - Расстояние в 3 раза до ближайшей станции метро = расстояние до ближайшей станции метро (единица измерения: метр)**

 **X4 number of convenience stores - Количество круглосуточных магазинов X4 = количество магазинов шаговой доступности в радиусе пешей доступности (целое число)**
 
 **X5 latitude - X5 latitude = географическая координата, широта. (единица измерения: градус)**
 
 **X6 longitude - X6 долгота =географическая координата, долгота. (единица измерения: градус)**
 
 **Результат выглядит следующим образом**
 
 **Y house price of unit area - Y = цена за квадратный метр жилья (10000 новых тайваньских долларов/пин, где пин — местная единица измерения, 1 пин = 3,3 квадратного метра)**



**Для выполнения 9 лабораторной работы, необходимо - Постройте модель многомерной регрессии с использованием стратегии backward elimination.**

**Backward elimination — это метод отбора модели, который используется для уточнения статистической модели путём систематического удаления наименее значимых переменных.**

In [11]:
# Импорт необходимых библиотек
import numpy as np  # Работа с числовыми массивами и математическими операциями
import pandas as pd  # Работа с табличными данными
from sklearn.model_selection import train_test_split  # Разделение данных на обучающую и тестовую выборки
from sklearn.linear_model import LinearRegression  # Линейная регрессия
from sklearn.metrics import mean_squared_error, mean_absolute_error  # Метрики для оценки модели
import statsmodels.api as sm  # Библиотека для анализа статистической значимости признаков

# Загрузка данных
file_path = 'Real estate valuation data set.xlsx' 
dataset = pd.read_excel(file_path)  # Чтение данных из Excel-файла
print("Первые строки набора данных:")
print(dataset.head())  # Выводим первые 5 строк данных для проверки

# Выделение признаков и целевой переменной
# Используем все доступные числовые признаки, кроме целевой переменной
X = dataset.iloc[:, 1:-1]  # Все столбцы, кроме первого (ID) и последнего (целевая переменная)
y = dataset['Y house price of unit area']  # Целевая переменная — стоимость недвижимости

# Добавляем столбец единиц (константа) для оценки статистической значимости признаков в модели
X = sm.add_constant(X)

# Разделение выборки на обучающую и тестовую части
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Реализация стратегии Backward Elimination
# Инициализируем модель с полным набором признаков
X_opt = X_train.copy()  # Создаем копию обучающей выборки признаков
regressor_OLS = sm.OLS(y_train, X_opt).fit()  # Строим модель множественной линейной регрессии

# Пошаговое удаление признаков
print("Результаты начальной модели:")
summary_text = regressor_OLS.summary().as_text()  # Получаем текстовое представление summary
summary_text = summary_text.replace(
    "Notes:\n[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.",
    "Примечания:\n[1] Стандартные ошибки предполагают, что ковариационная матрица ошибок указана правильно."
).replace(
    "[2] The condition number is large, ", 
    "[2] Число обусловленности велико, "
).replace(
    ". This might indicate that there are\nstrong multicollinearity or other numerical problems.",
    ". Это может указывать на сильную мультиколлинеарность или другие численные проблемы."
)
print(summary_text)  # Печатаем переведенный текст

# Удаляем признаки с наибольшим p-value > 0.05, пока все оставшиеся признаки значимы
threshold = 0.05  # Уровень значимости
while True:
    max_p_value = max(regressor_OLS.pvalues)  # Находим максимальное p-value
    if max_p_value > threshold:  # Если оно превышает уровень значимости
        excluded_feature = regressor_OLS.pvalues.idxmax()  # Находим имя признака с максимальным p-value
        print(f"Удаляется признак: {excluded_feature} с p-value = {max_p_value:.3f}")
        X_opt = X_opt.drop(columns=[excluded_feature])  # Удаляем этот признак
        regressor_OLS = sm.OLS(y_train, X_opt).fit()  # Перестраиваем модель
    else:
        break  # Если все признаки значимы, завершаем процесс

# Вывод финальной модели
print("\nРезультаты финальной модели:")
summary_text = regressor_OLS.summary().as_text()
summary_text = summary_text.replace(
    "Notes:\n[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.",
    "Примечания:\n[1] Стандартные ошибки предполагают, что ковариационная матрица ошибок указана правильно."
).replace(
    "[2] The condition number is large, ", 
    "[2] Число обусловленности велико, "
).replace(
    ". This might indicate that there are\nstrong multicollinearity or other numerical problems.",
    ". Это может указывать на сильную мультиколлинеарность или другие численные проблемы."
)
print(summary_text)

# Обучение модели линейной регрессии с выбранными признаками
final_model = LinearRegression()
final_model.fit(X_opt, y_train)

# Предсказание тестовых данных
X_test_opt = X_test[X_opt.columns]  # Отбираем те же признаки, что остались в модели
y_pred = final_model.predict(X_test_opt)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
mae = mean_absolute_error(y_test, y_pred)  # Средняя абсолютная ошибка
print(f"\nСреднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")


Первые строки набора данных:
   No  X1 transaction date  X2 house age  \
0   1          2012.916667          32.0   
1   2          2012.916667          19.5   
2   3          2013.583333          13.3   
3   4          2013.500000          13.3   
4   5          2012.833333           5.0   

   X3 distance to the nearest MRT station  X4 number of convenience stores  \
0                                84.87882                               10   
1                               306.59470                                9   
2                               561.98450                                5   
3                               561.98450                                5   
4                               390.56840                                5   

   X5 latitude  X6 longitude  Y house price of unit area  
0     24.98298     121.54024                        37.9  
1     24.98034     121.53951                        42.2  
2     24.98746     121.54391                        47.3  
3

**Таким образом, в текущем наборе данных признаки, такие как X6 longitude не были статистически значимыми и были удалены, так как их p-value превышало заданный порог (0.05).**